In [1]:
# import packages
import pandas as pd
import math
import numpy as np
#import statsmodels.api as sm
from scipy import stats
import statistics
#import plotly.graph_objects as go
from scipy.stats import norm
#from plotly.subplots import make_subplots
#from sklearn.neighbors import KernelDensity
from scipy import optimize

In [2]:
# Load the market prices
df = pd.read_excel("/Users/dantoni/HW7/FIREDAT.xls", header=None)
df.columns = ['Date','DKK']

## Fitting a Generalized Pareto Distribution

### 1.

In [3]:
u1 = 10*10**6
u2 = 20*10**6

pos1 = df['DKK']>u1
z1 = df['DKK'][pos1]
pos2 = df['DKK']>u2
z2 = df['DKK'][pos2]

m1 = statistics.mean(z1)
S_21 = np.var(z1)
m2 = statistics.mean(z2)
S_22 = np.var(z2)

ksi1 = (1 - ((m1 - u1)**2)/S_21)/2
sigma_hat1 = ((m1 - u1) / 2) * (((m1 - u1)**2) / S_21 + 1)

ksi2 = (1 - ((m2 - u2)**2)/S_22)/2
sigma_hat2 = ((m2 - u2) / 2) * (((m2 - u2)**2) / S_22 + 1)

In [4]:
def GDPLogLikelihood(param, y):
    ksi_ = param[0]
    sigma_ = param[1]
    fct = (1 + (ksi_ * y)/sigma_)**-((1 + ksi_)/ ksi_)/sigma_
    return - np.sum(np.log(fct))

In [5]:
obs1 = z1.values - u1
obs2 = z2.values - u2

guesses1 = np.array([ksi1, sigma_hat1])
guesses2 = np.array([ksi2, sigma_hat2])

estimates_param1 = optimize.fmin(func = GDPLogLikelihood, x0 = guesses1, args = (obs1,))
estimates_param2 = optimize.fmin(func = GDPLogLikelihood, x0 = guesses2, args = (obs2,))

Optimization terminated successfully.
         Current function value: 2181.333358
         Iterations: 67
         Function evaluations: 154
Optimization terminated successfully.
         Current function value: 798.206316
         Iterations: 79
         Function evaluations: 175


In [6]:
ksi1_est, sigma1_est = estimates_param1
ksi2_est, sigma2_est = estimates_param2

In [7]:
print('For u =', u1, ': ')
print()
print('MM ksi / sigma: ',ksi1,'/',sigma_hat1, '\nMLE ksi / sigma: ',ksi1_est,'/', sigma1_est)
print()
print('For u =', u2, ': ')
print()
print('MM ksi / sigma: ',ksi2,'/',sigma_hat2, '\nMLE ksi / sigma: ',ksi2_est,'/', sigma2_est)

For u = 10000000 : 

MM ksi / sigma:  0.3679452428005136 / 8883665.258412506 
MLE ksi / sigma:  0.41483173557597164 / 8019108.368398156

For u = 20000000 : 

MM ksi / sigma:  0.34314657562860407 / 14580369.363902751 
MLE ksi / sigma:  0.5286889452917001 / 10954152.382569399


In [8]:
# We will use MM parameters

p1 = statistics.mean(df['DKK'] <= u1)
p2 = statistics.mean(df['DKK'] <= u2)

sigma_tilda1 = sigma_hat1 * (1 - p1)**ksi1
sigma_tilda2 = sigma_hat2 * (1 - p2)**ksi2

mu_tilda1 = u1 - (sigma_tilda1 * ((1 - p1)**-ksi1 - 1)) / ksi1
mu_tilda2 = u2 - (sigma_tilda2 * ((1 - p2)**-ksi2 - 1)) / ksi2

## Computing the VaR and ES

### 1.

In [9]:
p = 0.99

VaR1 = mu_tilda1 + sigma_tilda1/ksi1 * ((1 - p)**(-ksi1) - 1)
ES1 = VaR1 * (1 / (1 - ksi1) + (sigma_hat1 - ksi1 * u1) / ((1 - ksi1) * VaR1))


VaR2 = mu_tilda2 + sigma_tilda2/ksi2 * ((1 - p)**(-ksi2) - 1)
ES2 = VaR2 * (1 / (1 - ksi2) + (sigma_hat2 - ksi2 * u2) / ((1 - ksi2) * VaR2))

In [10]:
print('For u =', u1, ': ')
print()
print('VaR: ',VaR1, '\nES: ',ES1)
print()
print('For u =', u2, ': ')
print()
print('VaR: ',VaR2, '\nES: ',ES2)

For u = 10000000 : 

VaR:  35320366.77604337 
ES:  64115615.213478334

For u = 20000000 : 

VaR:  35765460.9461749 
ES:  66198785.27560148


### 2.

In [11]:
# With normal assumption and kernel estimation
losses = df['DKK']
mu = statistics.mean(losses)
sigma = statistics.variance(losses)

In [12]:
# Not sure if we have to take all the data set or just the one above the threshold
# Under Gaussian assumption
VaR_gaussian0 = mu + math.sqrt(sigma)*stats.norm.ppf(0.99)

In [13]:
print('Gaussian case : \nVaR: ',VaR_gaussian0)

Gaussian case : 
VaR:  25768910.005169176


In [14]:
def gaussianVaR(VaR, obs, bandwidth, alpha):
    return (np.mean(stats.norm.cdf((obs - VaR)/bandwidth))-alpha)**2

In [15]:
obs = np.array(losses)
bandwidth = pow(len(losses), -0.2) * pow(statistics.variance(losses), 0.5)
alpha = 0.01
guesses5 = VaR_gaussian0

In [16]:
estimates_Var0 = optimize.fmin(func = gaussianVaR, x0 = guesses5, args = (obs, bandwidth, alpha,))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 39
         Function evaluations: 78


In [17]:
print('Kernel case : \nVaR: ',estimates_Var0[0])

Kernel case : 
VaR:  34582394.083543375
